Iperparametri corrispondenti ai file con i pesi della rete, ottenuti dall'esecuzione di vari esperimenti 
- trained_model.pth --> rete 32-16, epsilon_decay=300000, gamma=0.95, buffer_capacity=100000
- trained_model1.pth --> rete 64-32, epsilon_decay=150000, gamma=0.95, buffer_capacity=100000 (best up to now)
- trained_model2.pth --> rete 64-32, epsilon_decay=175000, gamma=0.95, buffer_capacity=100000
- trained_model3.pth --> rete 64-32, epsilon_decay=150000, gamma=0.99, buffer_capacity=100000 (looping) XXXXXX
- trained_model4.pth --> rete 64-32, epsilon_decay=150000, gamma=0.95, buffer_capacity=20000 (looping) XXXXXXX
- Observation space cambiato: [head_x, head_y, food_dx, food_dy, dist_up, dist_down, dist_left, dist_right, surr_up, surr_down, surr_left, surr_right, tail_dx, tail_dy, dir_x, dir_y] --> 16 componenti invece che 8
    - https://gemini.google.com/share/a3c736821d16
    - trained_model5.pth --> rete 64-32, epsilon_decay=150000, gamma=0.95, buffer_capacity=20000, lr=1e-4, target_update_freq=1000, batch_size=64

In [1]:
import gym_snakegame
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import matplotlib.pyplot as plt
import time
from collections import deque

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cpu


In [2]:
class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, action_dim)
        )

    def forward(self, x):
        return self.net(x)

def select_action(model, state, epsilon, act_dim):
    # Exploration: random action
    if random.random() < epsilon:
        return random.randrange(act_dim)
    # Exploitation: best action according to the network
    with torch.no_grad():
        # Inference: NN estimates action-value function
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(DEVICE)
        return int(torch.argmax(model(state)).item())

In [ ]:
def watch_agent_play(model_path, board_size=10, num_episodes=50):
    """
    Carica un modello e visualizza un numero specificato di partite.
    
    Args:
        model_path (str): Percorso del file .pth
        board_size (int): Dimensione della mappa
        num_episodes (int): Numero di partite da giocare
    """
    
    # 1. Configura l'ambiente
    env = gym.make(
        "gym_snakegame/SnakeGame-v0",
        board_size=board_size,
        n_channel=1,
        n_target=1,
        render_mode='human'
    )

    # 2. Ricalcola le dimensioni
    obs_dim = np.prod(env.observation_space.shape)
    action_dim = env.action_space.n

    # 3. Inizializza la rete e carica i pesi
    model = DQN(obs_dim, action_dim).to(DEVICE)
    
    try:
        model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    except FileNotFoundError:
        print(f"Errore: Il file '{model_path}' non è stato trovato.")
        return

    model.eval()
    print(f"Modello caricato da: {model_path}")
    print(f"Avvio di {num_episodes} partite di test...")

    all_rewards = []

    # --- CICLO DEGLI EPISODI ---
    for i_ep in range(num_episodes):
        state, _ = env.reset()
        done = False
        total_reward = 0
        
        print(f"\n--- Inizio Partita {i_ep + 1}/{num_episodes} ---")

        while not done:
            # Prepara lo stato
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(DEVICE)

            with torch.no_grad():
                q_values = model(state_tensor)
                action = q_values.argmax(dim=1).item()

            state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward
            
            # Opzionale: rallenta leggermente per rendere l'azione più visibile all'occhio umano
            # time.sleep(0.05) 

        # Fine dell'episodio corrente
        print(f"Partita {i_ep + 1} terminata! Reward totale: {total_reward}")
        all_rewards.append(total_reward)
        
        # Pausa breve tra una partita e l'altra
        time.sleep(1.0)

    env.close()
    
    # Statistiche finali
    mean_reward = np.mean(all_rewards)
    print("\n==================================")
    print(f"Sessione conclusa.")
    print(f"Partite giocate: {num_episodes}")
    print(f"Reward Medio: {mean_reward:.2f}")
    print(f"Miglior Partita: {np.max(all_rewards)}")
    print("==================================")

# --- ESECUZIONE ---
# Esempio: Visualizza 3 partite
watch_agent_play("trained_model5.pth", num_episodes=50)

C:\Users\utente\anaconda3\envs\rl-env\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Modello caricato da: trained_model5.pth
Avvio di 50 partite di test...


C:\Users\utente\anaconda3\envs\rl-env\Lib\site-packages\gymnasium\utils\passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
C:\Users\utente\anaconda3\envs\rl-env\Lib\site-packages\gymnasium\utils\passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")



--- Inizio Partita 1/50 ---
Partita 1 terminata! Reward totale: 19

--- Inizio Partita 2/50 ---
Partita 2 terminata! Reward totale: 19

--- Inizio Partita 3/50 ---
Partita 3 terminata! Reward totale: 18

--- Inizio Partita 4/50 ---
Partita 4 terminata! Reward totale: 18

--- Inizio Partita 5/50 ---
Partita 5 terminata! Reward totale: 28

--- Inizio Partita 6/50 ---
Partita 6 terminata! Reward totale: 6

--- Inizio Partita 7/50 ---
Partita 7 terminata! Reward totale: 15

--- Inizio Partita 8/50 ---
Partita 8 terminata! Reward totale: 20

--- Inizio Partita 9/50 ---
